# Case Study: Backend

This notebook contains the backend code for the app. We first load the model you've trained (and saved as `"strength_model.pkl"`) in the previous notebook `a` and then make the predictions and optimize the water content of the concrete mixture. 

The backend is built using [FastAPI](https://fastapi.tiangolo.com/), a modern, fast (high-performance), web framework for building APIs with Python. Please note that normally this code would be run as a regular script, not in a notebook.

In [ ]:
import numpy as np
import pandas as pd
import joblib
import nest_asyncio
import uvicorn
from fastapi import Body, FastAPI
from pydantic import BaseModel, Field
from scipy.optimize import minimize

In [ ]:
# load your trained model - make sure it's saved under the right name!
model = joblib.load("strength_model.pkl")

In [ ]:
# configure the host and port where the server will run
# host depends on your setup -> "127.0.0.1" if running locally, "0.0.0.0" inside a docker container
host = "127.0.0.1"
port = 8000

In [ ]:
# helper function to compute the optimized water content
def optimize_water(model, x, value_min=120., value_max=250., target_strength=42.5):
    """
    Optimize the water content for a concrete mixture.
    
    Inputs:
        - model: the trained model
        - x: pandas dataframe row with one data point
        - value_min: minimum bound for water content (default: 120.)
        - value_max: maximum bound for water content (default: 250.)
        - target_strength: what we would like the output to be (default: 42.5)
    Returns:
        - water_org: original water content
        - water_new: optimized water content
        - pred_org: original strength prediction of the model
        - pred_new: strength prediction with optimized water content
    """
    # original situation
    water_org = x["water"].values[0]
    pred_org = model.predict(x)[0]
    print(f"original prediction with water content {water_org:.1f}: {pred_org:.2f} MPa")
    
    def _loss_fun(water_value):
        """
        Nested function (i.e., has access to all variables from the enclosing function)
        to compute the squared error between the models strength prediction with the given 
        water value and our target strength value.
        
        Inputs:
            - water_value: np.array with a single value, the proposed water content
        Returns:
            - loss: the squared error between the predicted and target strength
        """
        # insert the new value into our original data point
        new_x = x.copy()
        new_x["water"] = water_value[0]
        # predict strength with new water content
        pred_strength = model.predict(new_x)
        # optimization loss = squared difference to target value
        loss = (target_strength - pred_strength)**2
        return loss
    
    # use scipy's minimize function to find a value for 'water'
    # where the model predicts something close to our target value.
    # the start value for the optimization is the original water content.
    # to get realistic values, we additionaly specify bounds
    # based on the actual min/max values for the water content
    res = minimize(_loss_fun, np.array([water_org]), bounds=[(value_min, value_max)], method="Powell")
    # the optimized water content is stored in res.x (again a np.array)
    water_new = res.x[0]
    # check the final strength prediction
    new_x = x.copy()
    new_x["water"] = water_new
    pred_new = model.predict(new_x)[0]
    print(f"new prediction with water content {water_new:.1f}: {pred_new:.2f} MPa")
    return water_org, water_new, pred_org, pred_new

In [ ]:
# FastAPI app instance
app = FastAPI()

# pydantic model used for data validation
class ConcreteRecipe(BaseModel):
    cement: float = Field(ge=50., le=650., description="Cement amount in kg/m^3")
    slag: float = Field(ge=0., le=500., description="Slag amount in kg/m^3")
    fly_ash: float = Field(ge=0., le=300., description="Fly ash amount in kg/m^3")
    fine_aggregate: float = Field(ge=400., le=1100., description="Fine aggregate amount in kg/m^3")
    coarse_aggregate: float = Field(ge=500., le=1500., description="Coarse aggregate amount in kg/m^3")
    plasticizer: float = Field(ge=0., le=50., description="Plasticizer amount in kg/m^3")
    water: float = Field(ge=50., le=500., description="Water amount in kg/m^3")
    target_strength: float = Field(default=42.5, ge=0., le=100., description="Desired compressive strength in MPa")

# endpoint for the base URI, which can be queried with a GET request
@app.get("/")
def home():
    return "Congratulations! Your API is working."

# endpoint for /predict, which can be queried with a POST request to send the concrete recipe data
@app.post("/predict")
def predict_and_optimize(concrete_recipe: ConcreteRecipe):
    target_strength = concrete_recipe.target_strength
    # transform the given data into a pandas dataframe
    concrete_recipe = concrete_recipe.dict()
    features = ["cement", "slag", "fly_ash", "water", "plasticizer", "coarse_aggregate", "fine_aggregate"]
    x = pd.DataFrame({c: [concrete_recipe[c]] for c in features}, columns=features)
    # make predictions and get optimized water content
    water_org, water_new, pred_org, pred_new = optimize_water(model, x, target_strength=target_strength)
    # return the computed values as a JSON
    return {
        "water_org": water_org, 
        "water_new": water_new, 
        "pred_org": pred_org, 
        "pred_new": pred_new,
    }

By running the following cell you will spin up the server!

This causes the notebook to block (no cells/code can run) until you manually interrupt the kernel. You can do this by clicking on the Kernel tab and then on Interrupt or by entering Jupyter's command mode by pressing the `ESC` key and tapping the `I` key twice.

In [ ]:
# allow the server to be run in this interactive environment
nest_asyncio.apply()
# spin up the server  
uvicorn.run(app, host=host, port=port)